# Cleaning statement of vote files

Copyright and distributed under an [MIT License](https://opensource.org/licenses/MIT).

In [1]:
# Data processing libraries
import pandas as pd
import numpy as np

pd.options.display.max_columns = 200
idx = pd.IndexSlice

import json

## Background
Official precinct-level election results are available in the "Statement of Votes" (SOV) records from the [Boulder County Elections Division](https://www.bouldercounty.org/elections/by-year/) and [Colorado Secretary of State](https://www.coloradosos.gov/pubs/elections/Results/Archives.html) in Excel format.

Are the results of individual contests available at the level of precincts in an accessible format (XLS, CSV)?

| Year | Boulder | State |
| --- | --- | --- |
| 2022 | [✅](https://assets.bouldercounty.gov/wp-content/uploads/2022/12/2022G-Boulder-County-Official-Statement-of-Votes.xlsx) | ❌ |
| 2021 | [✅](https://assets.bouldercounty.gov/wp-content/uploads/2021/11/2021-Boulder-County-Coordinated-Election-Official-Statement-of-Votes-1.xlsx) | ❌ |
| 2020 | [✅](https://assets.bouldercounty.gov/wp-content/uploads/2020/11/2020-Boulder-County-General-Election-Official-Statement-of-Votes.xlsx) | [✅](https://www.coloradosos.gov/pubs/elections/Results/2020/2020GEPrecinctLevelResultsPosted.xlsx) |
| 2019 | [✅](https://assets.bouldercounty.gov/wp-content/uploads/2019/11/2019C-Official-Statement-Of-Votes-SOV.xls) | ❌ |
| 2018 | [✅](https://assets.bouldercounty.gov/wp-content/uploads/2018/11/2018-General-Election-Official-Statement-Of-Votes.xlsx) | [✅](https://www.coloradosos.gov/pubs/elections/Results/2018/2018GEPrecinctLevelResults.xlsx) |
| 2017 | [✅](https://assets.bouldercounty.gov/wp-content/uploads/2017/11/Results_SOV_Final.xlsx) | ❌ |
| 2016 | [✅](https://assets.bouldercounty.gov/wp-content/uploads/2017/03/2016-general-election-results-final-sov.xls) | [✅](https://www.coloradosos.gov/pubs/elections/Results/2016/General/2016GeneralTurnoutPrecinctLevel.xlsx) |
| 2015 | [✅](https://assets.bouldercounty.gov/wp-content/uploads/2017/03/2015-election-sov.xls) | ❌ |
| 2014 | [✅](https://assets.bouldercounty.gov/wp-content/uploads/2017/03/2014-general-election-sov.xls) | [✅](https://www.coloradosos.gov/pubs/elections/Results/2014/2014GeneralPrecinctTurnout.xlsx) |
| 2013 | [✅](https://assets.bouldercounty.gov/wp-content/uploads/2017/03/2013-election-sov.xls) | ❌ |
| 2012 | [✅](https://assets.bouldercounty.gov/wp-content/uploads/2017/03/2012-general-election-sov.xls) | [✅](https://www.coloradosos.gov/pubs/elections/Results/2012/2012GeneralPrecinctLevelTurnout.xlsx) |
| 2011 | [⚠️](https://assets.bouldercounty.gov/wp-content/uploads/2017/03/2011-election-sov.pdf) | ❌ |
| 2010 | [✅](https://assets.bouldercounty.gov/wp-content/uploads/2017/09/2010-general-sov.xls) | [✅](https://www.coloradosos.gov/pubs/elections/Results/2010/general/2010GeneralPrecinctTurnout.xlsx) |
| 2009 | ❌ | ❌ |
| 2008 | [✅](https://assets.bouldercounty.gov/wp-content/uploads/2017/12/2008-general-election-sov.xls) | [✅](https://www.coloradosos.gov/pubs/elections/Results/2008/2008GeneralPrecinctTurnout.xlsx) |
| 2007 | ❌ | ❌ |
| 2006 | ❌ | [✅](https://www.coloradosos.gov/pubs/elections/Results/2006/2006GeneralPrecinctBallotsCast.xlsx) | 
| 2005 | ❌ | ❌ |
| 2004 | ❌ | [✅](https://www.coloradosos.gov/pubs/elections/Results/2004/2004GeneralPrecinctBallotsCast.xlsx) |

Load up a list of precinct numbers within the City of Boulder.

In [20]:
with open('cob_precincts.json','r') as f:
    cob_precincts = json.load(f)

## Boulder County Clerk data

In [37]:
boco_original_path = './Original Data/Boulder County/'
boco_cleaned_path = './Cleaned Data/Boulder County/'

common_cols = ['Active Voters','Total Ballots','Contest Name','Choice Name','Total Votes','Party','Precinct Name']

### 2022

In [67]:
# Read in file
sov22_df = pd.read_excel(
    boco_original_path + '2022-general-sov.xlsx',
    dtype={'Precinct Name \n(Long)':str}
)

# Rename columns for consistency
sov22_df = sov22_df.rename(
    columns = {
        'Contest Title':'Contest Name',
        'Precinct Name \n(Short)':'Precinct Name (Short)',
        'Precinct Name \n(Long)':'Precinct Name',
        'Active \nVoters':'Active Voters',
        'Total \nBallots':'Total Ballots',
        'Total \nVotes':'Total Votes',
        'Total \nUndervotes':'Total Undervotes', 
        'Total \nOvervotes':'Total Overvotes'
    }
)

# Compute fraction of votes and turnout
sov22_df['Fraction'] = sov22_df['Total Votes']/sov22_df['Total Ballots']
sov22_df['Turnout'] = sov22_df['Total Votes']/sov22_df['Active Voters']

# Subset to common columns
sov22_df = sov22_df.reindex(columns=common_cols)

# Write common columns to disk
sov22_df.to_csv(
    boco_cleaned_path + '2022-general-sov.csv',
    index = False,
    encoding='utf8'
)

# Inspect
sov22_df.head(2)

,Active Voters,Total Ballots,Contest Name,Choice Name,Total Votes,Party,Precinct Name
0,1398.0,1030,United States Senator,Michael Bennet,664,DEM,"2151907002, 2171907636"
1,1398.0,1030,United States Senator,Joe O'Dea,340,REP,"2151907002, 2171907636"


### 2021

In [66]:
sov21_df = pd.read_excel(
    boco_original_path + '2021-general-sov.xlsx',
    dtype={'Precinct Name':str}
)

sov21_df = sov21_df.rename(
    columns = {
        'Contest Title':'Contest Name'
    }
)

sov21_df['Fraction'] = sov21_df['Total Votes']/sov21_df['Total Ballots']
sov21_df['Turnout'] = sov21_df['Total Votes']/sov21_df['Active Voters']

# Subset to common columns
sov21_df = sov21_df.reindex(columns=common_cols)

# Write common columns to disk
sov21_df.to_csv(
    boco_cleaned_path + '2021-general-sov.csv',
    index = False,
    encoding='utf8'
)

# Inspect
sov21_df.head(2)

,Active Voters,Total Ballots,Contest Name,Choice Name,Total Votes,Party,Precinct Name
0,441,263,City of Boulder Council Candidates,Michael Christy,127,NaN,2181007800
1,441,263,City of Boulder Council Candidates,Mark Wallach,126,NaN,2181007800


### 2020

In [65]:
sov20_df = pd.read_excel(
    boco_original_path + '2020-general-sov.xlsx',
    dtype={'Precinct Name':str}
)

sov20_df = sov20_df.rename(
    columns = {
        'Active\nVoters':'Active Voters'
    }
)

sov20_df['Fraction'] = sov20_df['Total Votes']/sov20_df['Total Ballots']
sov20_df['Turnout'] = sov20_df['Total Votes']/sov20_df['Active Voters']

# Subset to common columns
sov20_df = sov20_df.reindex(columns=common_cols)

# Write common columns to disk
sov20_df.to_csv(
    boco_cleaned_path + '2020-general-sov.csv',
    index = False,
    encoding='utf8'
)

# Inspect
sov20_df.head(2)

,Active Voters,Total Ballots,Contest Name,Choice Name,Total Votes,Party,Precinct Name
0,1547,1399,Presidential Electors,Joseph R. Biden / Kamala D. Harris,997,DEM,2163307100
1,1547,1399,Presidential Electors,Donald J. Trump / Michael R. Pence,355,REP,2163307100


### 2019

In [64]:
sov19_df = pd.read_excel(
    boco_original_path + '2019-general-sov.xls',
    dtype={'Precinct Name':str}
)

sov19_df = sov19_df.rename(
    columns = {
        'Contest Title':'Contest Name'
    }
)

sov19_df['Fraction'] = sov19_df['Total Votes']/sov19_df['Total Ballots']
sov19_df['Turnout'] = sov19_df['Total Votes']/sov19_df['Active Voters']

# Subset to common columns
sov19_df = sov19_df.reindex(columns=common_cols)

# Write common columns to disk
sov19_df.to_csv(
    boco_cleaned_path + '2019-general-sov.csv',
    index = False,
    encoding='utf8'
)

# Inspect
sov19_df.head(2)

,Active Voters,Total Ballots,Contest Name,Choice Name,Total Votes,Party,Precinct Name
0,428,266,City of Boulder Council Candidates,Brian Dolan,94,NaN,2181007800
1,428,266,City of Boulder Council Candidates,Rachel Friend,97,NaN,2181007800


### 2018

In [63]:
sov18_df = pd.read_excel(
    boco_original_path + '2018-general-sov.xlsx',
    dtype={'Precinct Name':str}
)

sov18_df = sov18_df.rename(
    columns = {
        'Contest Title':'Contest Name'
    }
)

sov18_df['Fraction'] = sov18_df['Total Votes']/sov18_df['Total Ballots']
sov18_df['Turnout'] = sov18_df['Total Votes']/sov18_df['Active Voters']

# Subset to common columns
sov18_df = sov18_df.reindex(columns=common_cols)

# Write common columns to disk
sov18_df.to_csv(
    boco_cleaned_path + '2018-general-sov.csv',
    index = False,
    encoding='utf8'
)

# Inspect
sov18_df.head(2)

,Active Voters,Total Ballots,Contest Name,Choice Name,Total Votes,Party,Precinct Name
0,1377,1149,Representative to the United States Congress -...,Joe Neguse,723,DEM,2163307100
1,1377,1149,Representative to the United States Congress -...,Peter Yu,337,REP,2163307100


### 2017

In [62]:
sov17_df = pd.read_excel(
    boco_original_path + '2017-general-sov.xlsx',
    dtype={'Precinct Name':str}
)

sov17_df = sov17_df.rename(
    columns = {
        'Contest Title':'Contest Name'
    }
)

sov17_df['Fraction'] = sov17_df['Total Votes']/sov17_df['Total Ballots']
sov17_df['Turnout'] = sov17_df['Total Votes']/sov17_df['Active Voters']

# Subset to common columns
sov17_df = sov17_df.reindex(columns=common_cols)

# Write common columns to disk
sov17_df.to_csv(
    boco_cleaned_path + '2017-general-sov.csv',
    index = False,
    encoding='utf8'
)

# Inspect
sov17_df.head(2)

,Active Voters,Total Ballots,Contest Name,Choice Name,Total Votes,Party,Precinct Name
0,427,230,City of Boulder Council Candidates,Bill Rigler,73,NaN,2181007800
1,640,209,City of Boulder Council Candidates,Bill Rigler,43,NaN,2181007801


### 2016

In [61]:
sov16_df = pd.read_excel(
    boco_original_path + '2016-general-sov.xls',
    dtype={'Precinct Name':str}
)

sov16_df = sov16_df.rename(
    columns = {
        'Total\nBallots':'Total Ballots',
        'Total\nVotes':'Total Votes',
        'Contest Title':'Contest Name',
        'Active\nVoters':'Active Voters',
        'Total\nUnder\nVotes':'Total Under Votes',
        'Total\nOver\nVotes':'Total Over Votes'
    }
)

sov16_df['Fraction'] = sov16_df['Total Votes']/sov16_df['Total Ballots']
sov16_df['Turnout'] = sov16_df['Total Votes']/sov16_df['Active Voters']

# Subset to common columns
sov16_df = sov16_df.reindex(columns=common_cols)

# Write common columns to disk
sov16_df.to_csv(
    boco_cleaned_path + '2016-general-sov.csv',
    index = False,
    encoding='utf8'
)

# Inspect
sov16_df.head(2)

,Active Voters,Total Ballots,Contest Name,Choice Name,Total Votes,Party,Precinct Name
0,1291.0,1174,Presidential Electors,Hillary Clinton / Tim Kaine,724,NaN,2163307100
1,1291.0,1174,Presidential Electors,Donald J. Trump / Michael R. Pence,327,NaN,2163307100


### 2015

In [60]:
sov15_df = pd.read_excel(
    boco_original_path + '2015-general-sov.xls',
    dtype={'Precinct Name':str}
)

# Replace bizarre dual-precinct precinct
sov15_df = sov15_df.replace({'Precinct Name':{'2181007800, 2181207403':'2181007800'}})

sov15_df = sov15_df.rename(
    columns = {
        'Contest Title':'Contest Name'
    }
)

sov15_df['Fraction'] = sov15_df['Total Votes']/sov15_df['Total Ballots']
sov15_df['Turnout'] = sov15_df['Total Votes']/sov15_df['Active Voters']

# Subset to common columns
sov15_df = sov15_df.reindex(columns=common_cols)

# Write common columns to disk
sov15_df.to_csv(
    boco_cleaned_path + '2015-general-sov.csv',
    index = False,
    encoding='utf8'
)

# Inspect
sov15_df.head(2)

,Active Voters,Total Ballots,Contest Name,Choice Name,Total Votes,Party,Precinct Name
0,417,223,City of Boulder Council Candidates,Jared Kaszuba,16,NaN,2181007800
1,417,223,City of Boulder Council Candidates,Lisa Morzel,59,NaN,2181007800


### 2014

In [59]:
sov14_df = pd.read_excel(
    boco_original_path + '2014-general-sov.xls',
    dtype={'Precinct \nName':str}
)

sov14_df = sov14_df.rename(
    columns = {
        'Precinct \nName':'Precinct Name',
        'Total \nBallots':'Total Ballots',
        'Total \nVotes':'Total Votes',
        'Contest Title':'Contest Name'
    }
)

sov14_df['Fraction'] = sov14_df['Total Votes']/sov14_df['Total Ballots']
sov14_df['Turnout'] = sov14_df['Total Votes']/sov14_df['Active Voters']

# Subset to common columns
sov14_df = sov14_df.reindex(columns=common_cols)

# Write common columns to disk
sov14_df.to_csv(
    boco_cleaned_path + '2014-general-sov.csv',
    index = False,
    encoding='utf8'
)

# Inspect
sov14_df.head(2)

,Active Voters,Total Ballots,Contest Name,Choice Name,Total Votes,Party,Precinct Name
0,1118,827,United States Senator,Mark Udall,467,DEM,2163307100
1,1118,827,United States Senator,Cory Gardner,314,REP,2163307100


### 2013

In [58]:
sov13_df = pd.read_excel(
    boco_original_path + '2013-general-sov.xls',
    sheet_name='SOV',
    dtype={'Precinct Name':str}
)

sov13_df = sov13_df.rename(
    columns = {
        'Contest Title':'Contest Name'
    }
)

sov13_df['Contest Name'] = sov13_df['Contest Name'].str.title()
sov13_df['Fraction'] = sov13_df['Total Votes']/sov13_df['Total Ballots']
sov13_df['Turnout'] = sov13_df['Total Votes']/sov13_df['Active Voters']

# Subset to common columns
sov13_df = sov13_df.reindex(columns=common_cols)

# Write common columns to disk
sov13_df.to_csv(
    boco_cleaned_path + '2013-general-sov.csv',
    index = False,
    encoding='utf8'
)

# Inspect
sov13_df.head(2)

,Active Voters,Total Ballots,Contest Name,Choice Name,Total Votes,Party,Precinct Name
0,426,249,City Of Boulder Council Candidates,Mary Young,66,NaN,2181007800
1,426,249,City Of Boulder Council Candidates,John Gerstle,58,NaN,2181007800


### 2012
2012 takes a **lot** of effort to clean up.

In [21]:
sov12_raw_df = pd.read_excel(
    boco_original_path + '2012-general-sov.xls',
    header=3
)

c0 = sov12_raw_df.isnull().sum(1) == 16
c1 = sov12_raw_df.iloc[:,0] != 'Continued'

sov12_title_indices = sov12_raw_df[c0 & c1].index

Presidental electors are spread across three panels, so this needs special care.

In [22]:
# First panel of presidental electors
panel_df0 = sov12_raw_df.iloc[4:236].replace({'%':np.nan}).dropna(how='all',axis=1)
panel_df0.columns = sov12_raw_df.loc[2,panel_df0.columns].str.replace('\r\n',' ').str.strip()

# Second panel of presidential electors, ignoring redundant columns
panel_df1 = sov12_raw_df.iloc[243:475].replace({'%':np.nan}).dropna(how='all',axis=1)
panel_df1.columns = sov12_raw_df.loc[241,panel_df1.columns].str.replace('\r\n',' ').str.strip()
cleaned_panel_df1 = panel_df1.iloc[:,[0,7,8,9,10,11,12,13]]

# Third panel of presidential electors, ignoring redundant columns
panel_df2 = sov12_raw_df.iloc[482:714].replace({'%':np.nan}).dropna(how='all',axis=1)
panel_df2.columns = sov12_raw_df.loc[480,panel_df2.columns].str.replace('\r\n',' ').str.strip()
cleaned_panel_df2 = panel_df2.iloc[:,[0,7,8,9]]

# Join first and second panels together on Precinct
sov12_presidential = pd.merge(
    left = panel_df0,
    right = cleaned_panel_df1,
    left_on = 'Precinct',
    right_on = 'Precinct',
    how = 'outer'
)

# Join joined panels with third panel on Precinct
sov12_presidential = pd.merge(
    left = sov12_presidential,
    right = cleaned_panel_df2,
    left_on = 'Precinct',
    right_on = 'Precinct',
    how = 'outer'
)

# Melt wide data to tidy data
sov12_pres_melted = pd.melt(
    frame = sov12_presidential,
    id_vars = ['Precinct','Total Ballots Cast','Registered Voters'],
    value_vars = sov12_presidential.columns[7:],
    var_name = 'Choice Name',
    value_name = 'Total Votes'
)

# Add Contest Name
sov12_pres_melted['Contest Name'] = 'Presidential Electors'

# sov12_pres_melted.head()

Extract the data for the other contest panels.

In [23]:
# Dictionary to store results for later concatenation
cleaned_panels = {}

# Define the panel ranges for other contests
panel_ranges = sov12_raw_df.iloc[sov12_title_indices,0].to_dict()

# Loop through other panels
for i,(k,v) in enumerate(panel_ranges.items()):
    
    # Ignore the first three panels corresponding to presidential electors
    if i > 2:
        
        # Name of contest is first row of panel
        contest_name = sov12_raw_df.iloc[k,0]
        
        # Row corresponding to panel names
        columns = k+2
        
        # The data starts at this row
        data_start = k+4
        
        # If we're not in the last panel, the data stops two rows before the next panel
        if i + 1 < len(panel_ranges):
            data_stop = list(panel_ranges.keys())[i+1] - 2
        else:
            data_stop = sov12_raw_df.index.max()
        
        # Slice the rows with data, drop blank columns
        panel_df = sov12_raw_df.iloc[data_start:data_stop].replace({'%':np.nan}).dropna(how='all',axis=1)
        
        # Rename the columns based on the column row
        panel_df.columns = sov12_raw_df.loc[columns,panel_df.columns].str.replace('\r\n',' ').str.strip()
        
        # Melt the wide data down to tidy data
        melted_df = pd.melt(
            frame = panel_df,
            id_vars = ['Precinct','Total Ballots Cast','Registered Voters'],
            value_vars = panel_df.columns[7:],
            var_name = 'Choice Name',
            value_name = 'Total Votes'
        )

        # Store in the dictionary
        cleaned_panels[contest_name.title()] = melted_df

Concatenate the data together.

In [24]:
# Concatenate the cleaned panels together, ignore rows with totals
sov12_cleaned_df = pd.concat(cleaned_panels).reset_index(0)
sov12_cleaned_df = sov12_cleaned_df[sov12_cleaned_df['Choice Name'] != 'Totals']
sov12_cleaned_df.rename(columns={'level_0':'Contest Name'},inplace=True)

# Add in the presidential elector data
sov12_cleaned_df = pd.concat([sov12_pres_melted,sov12_cleaned_df])

# Reorder the columns
sov12_cleaned_df = sov12_cleaned_df[['Precinct','Registered Voters','Total Ballots Cast','Contest Name','Choice Name','Total Votes']]

# Reset the index
sov12_cleaned_df.reset_index(drop=True,inplace=True)

In [25]:
# Find the Choice Names containing 'Democrat' or 'Republican'
dem_contests = sov12_cleaned_df.groupby('Contest Name').agg({'Choice Name':'unique'})['Choice Name'].apply(lambda x:bool([i for i in x if 'Democrat' in i]))
rep_contests = sov12_cleaned_df.groupby('Contest Name').agg({'Choice Name':'unique'})['Choice Name'].apply(lambda x:bool([i for i in x if 'Republican' in i]))

# Find Contests with either a Democrat or Republican running
both_contests = dem_contests | rep_contests
partisan_contests = both_contests[both_contests]

# Subset to the partisan contests and split Choice from Party on 4 spaces
choice_party = sov12_cleaned_df.loc[sov12_cleaned_df['Contest Name'].isin(partisan_contests.index),'Choice Name'].str.split('    ')
choice_s = choice_party.apply(lambda x:x[0])
party_s = choice_party.apply(lambda x:x[1])

# Create a new column for Party
sov12_cleaned_df['Party'] = np.nan

# Relabel the Choice Name on the candidate before the four spaces
sov12_cleaned_df.loc[choice_s.index,'Choice Name'] = choice_s.values

# Add the Party from after the four spaces
sov12_cleaned_df.loc[party_s.index,'Party'] = party_s.values

Add complete precinct values.

In [26]:
sov12_cleaned_df['Precinct'] = sov12_cleaned_df['Precinct'].astype(str)
sov12_cleaned_df.rename(columns = {'Precinct':'Precinct Short'},inplace=True)

sov12_cleaned_df = pd.merge(
    left = sov12_cleaned_df,
    right = sov21_df[['Precinct Name (Short)','Precinct Name']].drop_duplicates(),
    left_on = 'Precinct Short',
    right_on = 'Precinct Name (Short)',
    how = 'left'
)

sov12_cleaned_df = sov12_cleaned_df[[c for c in sov12_cleaned_df.columns if 'Short' not in c]]

# simple_precincts = ~(sov12_cleaned_df['Precinct'].str.contains(',') | sov12_cleaned_df['Precinct'].str.contains('Landowner'))

# sov12_cleaned_df.loc[simple_precincts.index,'Precinct'] = '2163307' + sov12_cleaned_df.loc[simple_precincts.index,'Precinct']

Rename columns to align with others.

In [57]:
sov12_cleaned_df.rename(
    columns = 
    {
        'Precinct':'Precinct Name',
        'Total Ballots Cast':'Total Ballots',
        'Registered Voters':'Active Voters'
    },
    inplace=True
)

sov12_cleaned_df['Fraction'] = sov12_cleaned_df['Total Votes']/sov12_cleaned_df['Total Ballots']
sov12_cleaned_df['Turnout'] = sov12_cleaned_df['Total Votes']/sov12_cleaned_df['Active Voters']

# Subset to common columns
sov12_df = sov12_cleaned_df.reindex(columns=common_cols)

# Write common columns to disk
sov12_df.to_csv(
    boco_cleaned_path + '2012-general-sov.csv',
    index = False,
    encoding='utf8'
)

# Inspect
sov12_df.head(2)

,Active Voters,Total Ballots,Contest Name,Choice Name,Total Votes,Party,Precinct Name
0,1369,1032,Presidential Electors,Virgil H. Goode Jr. / Jim Clymer,0,American,2163307100
1,1369,1040,Presidential Electors,Virgil H. Goode Jr. / Jim Clymer,2,American,2163307101


### Combine together
Concatenate and clean up.

In [69]:
all_sov_df = pd.concat({
    2012:sov12_df,
    2013:sov13_df,
    2014:sov14_df,
    2015:sov15_df,
    2016:sov16_df,
    2017:sov17_df,
    2018:sov18_df,
    2019:sov19_df,
    2020:sov20_df,
    2021:sov21_df,
    2022:sov22_df
})

# Clean up index
all_sov_df = all_sov_df.reset_index(0).reset_index(drop=True)

# Rename column
all_sov_df.rename(columns={'level_0':'Year'},inplace=True)

# Drop rows that have no ballots or voters
all_sov_df.dropna(subset=['Active Voters','Total Ballots'],inplace=True)

# Cast to proper types
all_sov_df['Active Voters'] = all_sov_df['Active Voters'].astype(int)
all_sov_df['Total Ballots'] = all_sov_df['Total Ballots'].astype(int)
all_sov_df['Total Votes'] = all_sov_df['Total Votes'].astype(int)
all_sov_df['Precinct Name'] = all_sov_df['Precinct Name'].astype(str)

# Number of rows
print("There are {0:,} rows.".format(len(all_sov_df)))

# Write to disk
all_sov_df.to_csv(
    boco_cleaned_path + 'all-general-sov.csv',
    index = False,
    encoding='utf8'
)

# Inspect
all_sov_df.head()

There are 158,093 rows.


,Year,Active Voters,Total Ballots,Contest Name,Choice Name,Total Votes,Party,Precinct Name
0,2012,1369,1032,Presidential Electors,Virgil H. Goode Jr. / Jim Clymer,0,American,2163307100
1,2012,1369,1040,Presidential Electors,Virgil H. Goode Jr. / Jim Clymer,2,American,2163307101
2,2012,1787,857,Presidential Electors,Virgil H. Goode Jr. / Jim Clymer,0,American,2163307102
3,2012,1186,954,Presidential Electors,Virgil H. Goode Jr. / Jim Clymer,1,American,2163307103
4,2012,1197,953,Presidential Electors,Virgil H. Goode Jr. / Jim Clymer,2,American,2163307104


## Feature engineering

### Filter to City of Boulder (CoB) precincts

In [56]:
# Precincts for 2021
precincts_21 = sov21_df.loc[sov21_df['Contest Name'] == 'City of Boulder Council Candidates','Precinct Name'].unique()
print("There are {0:,} total precincts in the {1} data.".format(len(precincts_21),2021))

# Precincts for 2019
precincts_19 = sov19_df.loc[sov19_df['Contest Name'] == 'City of Boulder Council Candidates','Precinct Name'].unique()
print("There are {0:,} total precincts in the {1} data.".format(len(precincts_19),2019))

# For 2017
precincts_17 = sov17_df.loc[sov17_df['Contest Name'] == 'City of Boulder Council Candidates','Precinct Name'].unique()
print("There are {0:,} total precincts in the {1} data.".format(len(precincts_17),2017))

# For 2015
precincts_15 = sov15_df.loc[sov15_df['Contest Name'] == 'City of Boulder Council Candidates','Precinct Name'].unique()
print("There are {0:,} total precincts in the {1} data.".format(len(precincts_15),2015))

# For 2013
precincts_13 = sov13_df.loc[sov13_df['Contest Name'] == 'City Of Boulder Council Candidates','Precinct Name'].unique()
print("There are {0:,} total precincts in the {1} data.".format(len(precincts_13),2013))

There are 88 total precincts in the 2021 data.
There are 88 total precincts in the 2019 data.
There are 87 total precincts in the 2017 data.
There are 87 total precincts in the 2015 data.
There are 87 total precincts in the 2013 data.


Identify the union of all precincts for CoB elections.

In [57]:
precints_union = sorted(list(set(precincts_13) | set(precincts_15) | set(precincts_17) | set(precincts_19)))
print("There are {0:,} total precincts for City of Boulder Council candidates".format(len(precints_union)))
precints_union[:5]

There are 88 total precincts for City of Boulder Council candidates


['2181007800', '2181007801', '2181007802', '2181007804', '2181007806']

Write out, because it's useful.

In [58]:
with open('cob_precincts.json','w') as f:
    json.dump(precints_union,f)

### Contests

Pivot table to find contests that were city-wide.

In [60]:
def citywide_pivot(sov_df,precincts_union,year):
    
    # Pivot precincts and contests to find contests with null values
    citywide_contests_pivot = pd.pivot_table(
        data = sov_df.loc[sov_df['Precinct Name'].isin(precints_union)],
        values = 'Total Votes',
        index = 'Precinct Name',
        columns = 'Contest Name',
    )
    
    # Drop columns (contests) with NaNs: not city-wide contests
    # citywide_contests = citywide_contests_pivot.dropna(axis=1,how='any').columns.tolist()
    null_precincts = citywide_contests_pivot.isnull().sum()
    citywide_contests = null_precincts[null_precincts <= 2].index.tolist()
    
    # Subset the statement of votes to City of Boulder precincts and contests
    c0 = sov_df['Precinct Name'].isin(precints_union)
    c1 = sov_df['Contest Name'].isin(citywide_contests)
    c2 = ~sov_df['Choice Name'].isin(["No/Against",'NO/AGAINST','No','Against the Measure','NO'])
    cob = sov_df[c0 & c1 & c2]

    # Pivot precincts by contest-choice with values as vote fraction
    cw_pivot = pd.pivot_table(
        data = cob,
        index = 'Precinct Name',
        columns = ['Contest Name','Choice Name'],
        values = ['Fraction'],
        fill_value = 0
    )
    
    # Add the year to columns to prevent duplicates, flatten multi-index column name
    cw_pivot = pd.concat([cw_pivot],keys=[year],names=['Year'],axis=1) # https://stackoverflow.com/a/42094658/1574687
    cw_pivot.columns = cw_pivot.columns.map(' '.join) # pandas multiindex flatten
    
    # Compute precinct-level turnout for each contest
    turnout = pd.pivot_table(
        data = cob,
        index = 'Precinct Name',
        columns = 'Contest Name',
        values = ['Total Ballots','Active Voters']
    )
    
    # Divide total ballots cast by number of active voters
    # turnout = turnout['Total Ballots'].div(turnout['Active Voters'])
    # turnout.columns = [year + ' turnout ' + c for c in turnout.columns]
    
    # Concatenate turnout to the contest-level voteshares
    # precinct_df = pd.concat([cw_pivot,turnout],axis=1).fillna(0)
    precinct_df = cw_pivot.fillna(0)
    
    return precinct_df

In [74]:
cw_pivot_21 = citywide_pivot(sov21_df,precints_union,'2021')
cw_pivot_20 = citywide_pivot(sov20_df,precints_union,'2020')
cw_pivot_19 = citywide_pivot(sov19_df,precints_union,'2019')
cw_pivot_18 = citywide_pivot(sov18_df,precints_union,'2018')
cw_pivot_17 = citywide_pivot(sov17_df,precints_union,'2017')
cw_pivot_16 = citywide_pivot(sov16_df,precints_union,'2016')
cw_pivot_15 = citywide_pivot(sov15_df,precints_union,'2015')
cw_pivot_14 = citywide_pivot(sov14_df,precints_union,'2014')
cw_pivot_13 = citywide_pivot(sov13_df,precints_union,'2013')
cw_pivot_12 = citywide_pivot(sov12_cleaned_df,precints_union,'2012')

cw_pivot_21.shape, cw_pivot_20.shape, cw_pivot_19.shape, cw_pivot_18.shape, cw_pivot_17.shape, cw_pivot_16.shape, cw_pivot_15.shape, cw_pivot_14.shape, cw_pivot_13.shape, cw_pivot_12.shape

((88, 28),
 (88, 72),
 (88, 26),
 (88, 75),
 (88, 29),
 (88, 81),
 (88, 33),
 (88, 67),
 (88, 31),
 (88, 56))

Horizontally concatenate.

In [75]:
results_df = pd.concat(
    objs = [
        cw_pivot_21,cw_pivot_20,cw_pivot_19,cw_pivot_18,cw_pivot_17,
        cw_pivot_16,cw_pivot_15,cw_pivot_14,cw_pivot_13,cw_pivot_12
    ],
    axis=1
)

results_df.columns = results_df.columns.to_flat_index()

results_df.to_csv('voting_results_2012_2021.csv')

results_df.head()

,2021 Fraction Amendment 78 (Constitutional) Yes/For,2021 Fraction Boulder Valley School District RE-2 Director District B (4 Years) Nicole Rajpal,2021 Fraction Boulder Valley School District RE-2 Director District B (4 Years) Sky Van Horn,2021 Fraction Boulder Valley School District RE-2 Director District B (4 Years) William Hamilton,2021 Fraction Boulder Valley School District RE-2 Director District E (4 Years) Beth Niznik,2021 Fraction Boulder Valley School District RE-2 Director District E (4 Years) Deann E. Bucher,2021 Fraction Boulder Valley School District RE-2 Director District E (4 Years) Kara Awaitha Frost,2021 Fraction Boulder Valley School District RE-2 Director District F (4 Years) Kitty Sargent,2021 Fraction CITY OF BOULDER BALLOT ISSUE 2I Yes/For,2021 Fraction CITY OF BOULDER BALLOT ISSUE 2J Yes/For,2021 Fraction City of Boulder Ballot Question 2K Yes/For,2021 Fraction City of Boulder Ballot Question 2L Yes/For,2021 Fraction City of Boulder Ballot Question 2M Yes/For,2021 Fraction City of Boulder Ballot Question 300 Yes/For,2021 Fraction City of Boulder Ballot Question 301 Yes/For,2021 Fraction City of Boulder Ballot Question 302 Yes/For,2021 Fraction City of Boulder Council Candidates Dan Williams,2021 Fraction City of Boulder Council Candidates David Takahashi,2021 Fraction City of Boulder Council Candidates Jacques Decalo,2021 Fraction City of Boulder Council Candidates Lauren Folkerts,2021 Fraction City of Boulder Council Candidates Mark Wallach,2021 Fraction City of Boulder Council Candidates Matt Benjamin,2021 Fraction City of Boulder Council Candidates Michael Christy,2021 Fraction City of Boulder Council Candidates Nicole Speer,2021 Fraction City of Boulder Council Candidates Steve Rosenblum,2021 Fraction City of Boulder Council Candidates Tara Winer,2021 Fraction Proposition 119 (Statutory) Yes/For,2021 Fraction Proposition 120 (Statutory) Yes/For,2020 Fraction Amendment 76 (Constitutional) YES/FOR,2020 Fraction Amendment 77 (Constitutional) YES/FOR,2020 Fraction Amendment B (Constitutional) YES/FOR,2020 Fraction Amendment C (Constitutional) YES/FOR,2020 Fraction Boulder County Commissioner - District 1 Cinda Kochen,2020 Fraction Boulder County Commissioner - District 1 Claire Levy,2020 Fraction Boulder County Commissioner - District 2 James T Crowder,2020 Fraction Boulder County Commissioner - District 2 Marta Loachamin,2020 Fraction Boulder County Court Judge - Jonathon P. Martin YES,2020 Fraction City Of Boulder Ballot Issue 2B YES/FOR,2020 Fraction City of Boulder Ballot Question 2C YES/FOR,2020 Fraction City of Boulder Ballot Question 2D YES/FOR,2020 Fraction City of Boulder Ballot Question 2E YES/FOR,2020 Fraction City of Boulder Ballot Question 2F YES/FOR,2020 Fraction Colorado Court of Appeals Judge - Craig R. Welling YES,2020 Fraction Colorado Court of Appeals Judge - Ted C. Tow III YES,2020 Fraction District Attorney-20th Judicial District Michael Dougherty,2020 Fraction District Court Judge - 20th Judicial District - Judith L. LaBuda YES,2020 Fraction District Court Judge - 20th Judicial District - Nancy Woodruff Salomone YES,2020 Fraction District Court Judge - 20th Judicial District - Patrick D. Butler YES,2020 Fraction District Court Judge - 20th Judicial District - Ross Macdonald YES,2020 Fraction District Court Judge - 20th Judicial District - Seftar Bakke YES,2020 Fraction Justice of the Colorado Supreme Court - Carlos A. Samour Jr. YES,2020 Fraction Justice of the Colorado Supreme Court - Melissa Hart YES,2020 Fraction Presidential Electors Alyson Kennedy / Malcolm Jarrett,2020 Fraction Presidential Electors Bill Hammons / Eric Bodenstab,2020 Fraction Presidential Electors Blake Huber / Frank Atwood,2020 Fraction Presidential Electors Brian Carroll / Amar Patel,2020 Fraction Presidential Electors Brock Pierce / Karla Ballard,2020 Fraction Presidential Electors Dario Hunter / Dawn Neptune Adams,2020 Fraction Presidential Electors Don Blankenship / William Mohr,2020 Fraction Presidential